# Read files

In [1]:
with open("input/05", "r") as fp:
    data = fp.read()[:-1]

In [2]:
test_txt = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [3]:
buckets = test_txt.split('\n\n')

In [73]:
buckets = data.split('\n\n')

---

In [4]:
def process_buckets(line):
    subs = line.split("\n")
    print(subs[0])
    
    lst = []
    for sub in subs[1:]:
        L = list(map(int, sub.split(" ")))
        lst.append(L)
        
    return lst
        

# Process for part 1

In [27]:
def convert_X_to_Y(v, L_maps):
    v1 = v
    for a, b, r in L_maps:
        if (v >= b) & (v < b+r):
            v1 = v - b + a
            
    return v1

In [30]:
seeds = list(map(int, buckets[0].split(" ")[1:]))
seeds

[79, 14, 55, 13]

In [10]:
L_maps = process_buckets(buckets[1])
L_maps

seed-to-soil map:


[[50, 98, 2], [52, 50, 48]]

In [14]:
for seed in seeds:
    

    x1 = convert_X_to_Y(seed, L_maps)
    print(x1)

81
14
57
13


### All maps

In [24]:
LL_maps = []
for buck in buckets[1:]:
    LL_maps.append(process_buckets(buck))

seed-to-soil map:
soil-to-fertilizer map:
fertilizer-to-water map:
water-to-light map:
light-to-temperature map:
temperature-to-humidity map:
humidity-to-location map:


In [25]:
results = seeds.copy()
for L_maps in LL_maps:
    results = list(map(lambda x: convert_X_to_Y(x, L_maps), results))
    print(results)

[2693474491, 12212989, 2916488878, 2285625439, 2434574552, 8776260, 1224405006, 368941186, 2152043730, 124877531, 874771481, 36057332, 3422399727, 159289722, 1152623119, 106230212, 2249621993, 159131132, 621215889, 4213184]
[4071845592, 1767780175, 2763179843, 2680082804, 3049303741, 1764343446, 1015620994, 1615067414, 3805339953, 760704424, 1447643120, 1791624518, 3577464347, 389675167, 943839107, 1928223772, 2644079358, 389516577, 259700961, 1759780370]
[3213864440, 2943544494, 322827787, 2921276310, 608951685, 1595294430, 1879981248, 3603629601, 3450917956, 1964046124, 2600843009, 2967388837, 1137112291, 3997610143, 2545174038, 2102680285, 2885272864, 3997451553, 3867635937, 1590731354]
[2239634747, 3092720947, 1194408900, 3070452763, 1331750351, 456470919, 2563932206, 219461780, 66750135, 2036433759, 2672782562, 3116565290, 1749507200, 760269115, 2617113591, 1679278480, 3563832749, 760110525, 630294909, 451907843]
[1419679783, 672099772, 4111615129, 649831588, 4248956580, 285121916

In [26]:
print("Part 1: {}".format(min(results)))

Part 1: 178159714


# Part 2

Find the lowest seed number.
Need to work with ranges.
Cut ranges if needed

## Process buckets

In [74]:
LL_maps = []
for buck in buckets[1:]:
    LL_maps.append(process_buckets(buck))

seed-to-soil map:
soil-to-fertilizer map:
fertilizer-to-water map:
water-to-light map:
light-to-temperature map:
temperature-to-humidity map:
humidity-to-location map:


## Process seeds

In [75]:
seeds = list(map(int, buckets[0].split(" ")[1:]))
l = len(seeds) / 2
lst_rg = []
for i, j in zip(seeds[::2], seeds[1::2]):
    lst_rg.append([i, i+j])


In [79]:
lst_rg # Last value excluded

[[1347397244, 1359610233],
 [2916488878, 3951005553],
 [2821376423, 2830152683],
 [2240804122, 2609745308],
 [824872000, 949749531],
 [1597965637, 1634022969],
 [4091290431, 4250580153],
 [1875817275, 1982047487],
 [998513229, 1157644361],
 [2671581775, 2675794959]]

## Cut into ranges

In [12]:
def convert_X_to_Y(v, L_maps):
    v1 = v
    for a, b, r in L_maps:
        """
        b----------b+r
        |    v      |
        => convert
        
        """
        if (v >= b) & (v < b+r):
            v1 = v - b + a
            
    return v1

In [67]:
def convert_RX_RY(rg, L_maps):
    """rg: [a, b[
    """
    lst_rg = [rg]
    lst_final = []
    """
            b------b+r
    xxxx
                           xxxx
        xxxyyyy
                 yyyyxxxx
             yyyyyy
        xxxyyyyyyyyyyxxxxxxx
    
    """
    
    while lst_rg != []:
        ai, bi = lst_rg.pop()
        
        FLAG = True
        for a, b, r in L_maps:
            fx = lambda x: x - b + a 
        
            if (bi <= b) | (ai >= b+r): # Completely outside
                continue
                
            elif (ai < b) & (b + r < bi):
                lst_final.append([fx(b), fx(b+r)])
                lst_rg.append([ai, b])
                lst_rg.append([b+r, bi])
                FLAG = False
                break
            
            elif (b <= ai) & (bi <= b + r): # Fully included
                lst_final.append([fx(ai), fx(bi)])
                # Stop the loop as both are contained / processed
                FLAG = False
                break
                
            elif (b <= ai) & (b + r < bi): # Partial overlap with (b + r < bi)
                lst_final.append([fx(ai), fx(b+r)])
                lst_rg.append([b+r, bi])
                FLAG = False
                break
            
            elif (bi <= b + r): # Partial overlap (ai < b)
                
                lst_final.append([fx(b), fx(bi)])
                lst_rg.append([ai, b])
                FLAG = False
                break
                
            else:
                print("Error, case not covered")
        
        if FLAG:
            lst_final.append([ai, bi])
        
        
            
    return lst_final

In [80]:
results = lst_rg.copy()
print(results)
print("="*40)

for idx, L_maps in enumerate(LL_maps):
    print(idx, len(results))
    R = []
    for res in results:
        R.extend(convert_RX_RY(res, L_maps))
        
    results = R
    
    #print(idx, results)

[[1347397244, 1359610233], [2916488878, 3951005553], [2821376423, 2830152683], [2240804122, 2609745308], [824872000, 949749531], [1597965637, 1634022969], [4091290431, 4250580153], [1875817275, 1982047487], [998513229, 1157644361], [2671581775, 2675794959]]
0 10
1 23
2 43
3 56
4 82
5 99
6 118


In [81]:
print("Part 2: ", min(list(map(lambda x: x[0], results))))

Part 2:  100165128
